#### 1. 搭建的神经网络，使用olivettiface数据集进行训练

In [71]:
import torch
import torch.nn as nn
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split



# 定义超参数
LR = 1e-3
epochs = 1000 #训练轮次，100轮左右损失率降到了1以下，1000轮降到0.3左右，Accuracy: 76.03%
BATCH_SIZE = 10  #每批128张


# 1.生成随机样本
X, y = fetch_olivetti_faces(data_home='./face_data',return_X_y=True,shuffle=True)

# #拆分数据
# #局部样本训练模型（过拟合模型）
# #新样板数据模型表现不好（泛化能力差）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)


train_data_set = [(img,lbl) for img,lbl in zip(X_train, y_train)]
test_data_set = [(img,lbl) for img,lbl in zip(X_test, y_test)]


train_dl = torch.utils.data.DataLoader(train_data_set, batch_size=BATCH_SIZE, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_data_set, batch_size=BATCH_SIZE, shuffle=True)


FileNotFoundError: [Errno 2] No such file or directory: './face_data'

In [70]:
# 模型
model = nn.Sequential(
    nn.Linear(4096, 256),  #输入层
    nn.ReLU(),     #隐藏层
    nn.Linear(256, 40)  #输出层
)

#损失函数和优化器
loss_fn = nn.CrossEntropyLoss() #交叉熵损失函数

#优化器（模型参数更新）
optimizer = torch.optim.SGD(model.parameters(), lr=LR) #SGD优化器


for epoch in range(epochs):
    for data, target in train_dl:
        #前向运算
        output = model(data.reshape(-1, 4096))
        #计算损失
        loss = loss_fn(output, target) #计算梯度
        #反向传播
        optimizer.zero_grad() #所有参数梯度清零
        loss.backward()  #计算梯度（参数.grad）
        optimizer.step() #更新参数
    print(f'Epoch:{epoch}, Loss:{loss.item()}')

Epoch:0, Loss:3.7065036296844482
Epoch:1, Loss:3.654064893722534
Epoch:2, Loss:3.6609599590301514
Epoch:3, Loss:3.648454189300537
Epoch:4, Loss:3.6292781829833984
Epoch:5, Loss:3.6452877521514893
Epoch:6, Loss:3.717362880706787
Epoch:7, Loss:3.7067997455596924
Epoch:8, Loss:3.595839738845825
Epoch:9, Loss:3.6588339805603027
Epoch:10, Loss:3.6434338092803955
Epoch:11, Loss:3.583895206451416
Epoch:12, Loss:3.5366740226745605
Epoch:13, Loss:3.5868422985076904
Epoch:14, Loss:3.6083126068115234
Epoch:15, Loss:3.598557949066162
Epoch:16, Loss:3.6908276081085205
Epoch:17, Loss:3.565959930419922
Epoch:18, Loss:3.6196093559265137
Epoch:19, Loss:3.63334584236145
Epoch:20, Loss:3.5413050651550293
Epoch:21, Loss:3.531068801879883
Epoch:22, Loss:3.5967869758605957
Epoch:23, Loss:3.645918369293213
Epoch:24, Loss:3.576958417892456
Epoch:25, Loss:3.6204960346221924
Epoch:26, Loss:3.5988030433654785
Epoch:27, Loss:3.5447394847869873
Epoch:28, Loss:3.5513434410095215
Epoch:29, Loss:3.647212505340576
Epo

In [65]:
# 测试
correct = 0
total = 0

model.eval()  #train(false)
with torch.no_grad(): #不计算梯度
    for data, target in test_dl:
        output = model(data.reshape(-1, 4096))
        _, predicted = torch.max(output, 1) #返回第一个维度的最大值张量
        total += target.size(0) #返回张量的样本量
        correct += (predicted == target).sum().item()
print(f'Accuracy: {correct/total*100}%')


Accuracy: 95.83333333333334%


In [ ]:
import matplotlib.pyplot as plt

face = olivetti_faces.images[100]
plt.imshow(face, cmap='gray')
plt.show()

In [ ]:
# mac电脑，m1芯片
if torch.backends.mps.is_available():
    device = torch.device("mps")

device

device(type='mps')